In [1]:
from coosys import cartesian_to_kepler as ctk, cartesian_to_quaternion as ctq
from coosys import kepler_to_cartesian as ktc, quaternion_to_cartesian as qtc
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as LR
from sklearn.preprocessing import PolynomialFeatures as PF
from sklearn.model_selection import train_test_split as tts
from scipy.interpolate import UnivariateSpline as US
from scipy import signal
import spectrum
from tqdm.auto import tqdm
tqdm.pandas()

from matplotlib import pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras import models as M
from tensorflow.keras import layers as L
from tensorflow.keras import backend as K

D:\Anaconda3\envs\Standard\lib\site-packages\tqdm\std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
submission = pd.read_csv("submission.csv", encoding="utf8")
reference = pd.read_csv("idao_ans/tr1/ans.csv", encoding="utf8")
default = pd.read_csv("IDAO 2020/Track 1/test.csv", encoding="utf8")[
    ["id", "x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"]
].rename(columns={
    "x_sim": "x",
    "y_sim": "y",
    "z_sim": "z",
    "Vx_sim": "Vx",
    "Vy_sim": "Vy",
    "Vz_sim": "Vz",
})

In [3]:
def smape(y_true, y_pred):
    return 100 * (1 - np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred))))

def data_smape(reference, submission):
    columns = ["x", "y", "z", "Vx", "Vy", "Vz"]
    result = 0.
    N = submission.shape[0]
    for (_, row1), (_, row2) in tqdm(zip(reference.iterrows(), submission.iterrows()), total=N):
        result += smape(row1[columns].to_numpy(), row2[columns].to_numpy())
    return result / N

In [4]:
result = data_smape(reference, submission)
default_result = data_smape(reference, default)
print("Result: {} smape".format(result))
print("Default: {} smape".format(default_result))


Result: 65.10968141830337 smape
Default: 65.10968141830377 smape
